In [5]:
# %pip install numpy ultralytics opencv-contrib-python cvzone

In [6]:
# !pip install hydra-core>=1.2.0  matplotlib>=3.2.2  Pillow>=7.1.2  PyYAML>=5.3.1  requests>=2.23.0  scipy>=1.4.1  torch>=1.7.0  torchvision>=0.8.1  tqdm>=4.64.0  filterpy==1.4.5  scikit-image==0.19.3  lap==0.4.0

In [7]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
model = YOLO("yolov8l-pose.pt")

In [ ]:
from sort import *


In [ ]:
import os
import subprocess

In [ ]:

# !pip install protobuf==3.19


In [ ]:
# !pip install tensorflow==2.10


In [ ]:
from tensorflow.keras.models import load_model


In [ ]:

import os
import cv2
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
# Import the required libraries.
import os
import cv2
import math
import random
import datetime as dt
import tensorflow as tf
from collections import deque



In [ ]:
# import mediapipe as mp

# mp_pose = mp.solutions.pose
# mp_draw = mp.solutions.drawing_utils

# file_name = ""

In [ ]:
LRCN_model = load_model("LRCN_model__ano-4-_Date_Time_2024_01_03__15.h5",compile=False)

CLASSES_LIST = ['Fighting','Shoplifting','Explosion']


In [ ]:



to_model = {}

output={}

frame_width = 1280
frame_height = 720
cnt=0

def keyss(allkeys,x1, y1, x2, y2):
    for k in allkeys:
        all_within_box = all(
            x1 <= x <= x2  and
            y1 <= y <= y2 
            for x, y in k
        )

        if all_within_box:
            return k.tolist()
    return 0



i=0
res={}

cap = cv2.VideoCapture(r"E:\Jaiguru\downloads\consult\WhatsApp Video 2024-05-29 at 2.23.30 PM.mp4")

# folder_path = 'C:/OpenCV/consult/Frames/'+uploaded_video.filename
# if not os.path.exists(folder_path):
#     os.makedirs(folder_path)
#     print(f"Folder '{folder_path}' created successfully.")
# else:
#     print(f"Folder '{folder_path}' already exists.")

wCam, hCam = 640, 480
cap.set(3, wCam)
cap.set(4, hCam)
cap.set(cv2.CAP_PROP_FPS, 10)

model = YOLO("yolov8l-pose.pt")

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
            "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
            "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
            "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
            "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
            "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
            "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
            "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
            "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
            "teddy bear", "hair drier", "toothbrush"
            ]

# mask = cv2.imread("mask.png")

output_path = 'input2.mp4'

frame_width = int(cap.get(3))  # Width
frame_height = int(cap.get(4))  # Height

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 10, (frame_width, frame_height))



# Tracking

tracker = Sort(max_age=30, min_hits=3, iou_threshold=0.3)

frame_rate = 25

frame_skip = int(24 / frame_rate)  # Calculate the number of frames to skip

frame_number = 0 
preds = {}

while True:

    dummy={}
    success, img = cap.read()
    frame = img

    

    # if frame_number % 3 == 0:
    #     frame_number += 1
    #     continue
    # else:
    #     frame_number += 1
    if not success:
        print("Ignoring empty camera  frame.",i)
    # If loading a video, use 'break' instead of 'continue'.
        break
    # imgRegion = cv2.bitwise_and(img, mask)

    # imgGraphics = cv2.imread("graphics.png", cv2.IMREAD_UNCHANGED)
    # img = cvzone.overlayPNG(img, imgGraphics, (730, 260))

    # img2 = cv2.imread("C:/Users/Rajesh/Downloads/temp1.webp")

    results = model(img)

    # print(results)


    all_keypoints=[]  
    detections = np.empty((0, 5))

    for r in results:
        keypoints = r.keypoints
        for k in keypoints:
            # print("Key :" ,k.xy[0])
            all_keypoints.append(k.xy[0])
            
            
        boxes = r.boxes
        for box in boxes:
            # print("Box :", box.xyxy[0])
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1

            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            if currentClass == "person" and conf > 0.3:
                # cvzone.putTextRect(img, f'{currentClass} {conf}', (max(0, x1), max(35, y1)),
                #                    scale=0.6, thickness=1, offset=3)
                # cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=5)
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))


    # print("ALL Keypoints :",all_keypoints)
    resultsTracker = tracker.update(detections)
    img = results[0].plot()
    
    image = img
    
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        # print(result)
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1
        # cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                        scale=2, thickness=3, offset=10)
        # cvzone.putTextRect(img, "Normal", (max(0, x1), max(35, y1+50)),
        #                 scale=2, thickness=3, offset=10)
        


        # cx, cy = x1 + w // 2, y1 + h // 2
        # cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        # label=input("Enter "+str(id)+" :")
        a = {
            "person_id" : id ,
            "box" : [x1, y1, x2, y2] ,
            "keypoints" : keyss(all_keypoints,x1, y1, x2, y2),
            "label" : 0
        }
        dummy[str(id)] = a
        try : 

            if id not in to_model :
                to_model[id] = [cv2.resize(frame[y1: y2 , x1 : x2],(64,64))]+ [cv2.resize(frame[y1: y2 , x1 : x2],(64,64))]
            else :
                to_model[id].append(cv2.resize(frame[y1: y2 , x1 : x2],(64,64)))
                to_model[id].append(cv2.resize(frame[y1: y2 , x1 : x2],(64,64)))
        except :
            continue

    for k in to_model :
        if len(to_model[k]) == 200 :
            predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(to_model[k], axis = 0))[0]
            predicted_label = np.argmax(predicted_labels_probabilities)
            predicted_class_name = CLASSES_LIST[predicted_label]
            preds[id] = predicted_class_name
            to_model[k] = []

    for key in preds :
        cv2.putText(frame,  f"person id : {key}" + " Activity ::" +preds[key], (10, 30 + int(key*10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText( img ,  f"person id : {key}" + " Activity ::" +preds[key], (10, 30 + int(key*5)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    # if dummy == {}:
        # continue
    
    
    res[str(i)]=dummy   
    i+=1

    # Flip the image horizontally for a selfie-view display.
    # frame_resized = cv2.resize(img, (frame_width, frame_height))
    cnt+=1
    cv2.imshow("Image", img)
    cv2.imshow("Image5", frame)
    # # cv2.imshow("Image Output", opImg)
    out.write(img)


    # cv2.imshow("Image22", image)
    # cv2.imshow("ImageRegion", imgRegion)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
# with open('output15.json', 'w') as json_file:
#     json.dump(res, json_file, indent=2)

cap.release()
out.release()
cv2.destroyAllWindows()
output['message']=res
# resultsTracker = tracker.update(0)


# Define the command as a list of strings
# command = ['ffmpeg', '-y', '-i', 'C:/OpenCV/consult/input.mp4', '-vcodec', 'libx264', '-acodec', 'aac', 'C:/OpenCV/consult/output.mp4']

# Run the command
# subprocess.run(command)

tracker.reset()





0: 384x640 3 persons, 14.5ms
Speed: 0.8ms preprocess, 14.5ms inference, 8.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 25.3ms
Speed: 2.3ms preprocess, 25.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 30.1ms
Speed: 0.0ms preprocess, 30.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 22.5ms
Speed: 5.5ms preprocess, 22.5ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 26.0ms
Speed: 2.3ms preprocess, 26.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 24.5ms
Speed: 1.2ms preprocess, 24.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 26.8ms
Speed: 0.0ms preprocess, 26.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 32.3ms
Speed: 0.0ms preprocess, 32.3ms inference, 0.0ms postprocess per image at shape (

In [ ]:
cnt

1469

In [ ]:
preds

{1.0: 'Fighting', 12.0: 'Shoplifting', 21.0: 'Fighting'}

In [ ]:
# @app.route('/crop_video', methods=['POST'])
# def crop_video():
#     print("Got crop request !!!")
#     data = request.get_json()

#     global file_name 

#     # Path to your video file
#     video_file_path = f"C:/Users/Rajesh/Downloads/"+file_name

#     # Output folder to save cropped frames
#     output_folder = "C:/OpenCV/consult/Class/Theft_frames/"+file_name
#     output_folder11 = "C:/OpenCV/consult/Class/Theft_frames/"+file_name+"/full/"
#     output_folder2 = "C:/OpenCV/consult/Class/Normal_frames/"+file_name
#     output_folder22 = "C:/OpenCV/consult/Class/Normal_frames/"+file_name+"/full/"
#     os.makedirs(output_folder, exist_ok=True)
#     os.makedirs(output_folder2, exist_ok=True)
#     os.makedirs(output_folder11, exist_ok=True)
#     os.makedirs(output_folder22, exist_ok=True)
#     # os.mkdir("C:/OpenCV/consult/Class/Total_frames/")
#     # os.mkdir("C:/OpenCV/consult/Class/Total_frames/Normal_frames/")
#     # os.mkdir("C:/OpenCV/consult/Class/Total_frames/Theft_frames/")






#     # Open the video file
#     cap = cv2.VideoCapture(video_file_path)
#     cap.set(cv2.CAP_PROP_FPS, 10)

#     # Initialize a frame counter
#     frame_count = 0

#     # Loop through the frames
#     while True:
#         ret, frame = cap.read()
        
#         if not ret:
#             break  # End of video
        
#         frame_data = data.get(str(frame_count))
#         if frame_data:
#             for  person_data in frame_data.values():
#                 label = person_data["label"]
#                 if label == 1:
#                     # box = person_data["box"]
#                     # x1, y1, x2, y2 = box
#                     # cropped_frame = frame[y1:y2, x1:x2]
#                     # output_path = os.path.join(output_folder, f"{frame_count}_person.jpg")
#                     full = os.path.join(output_folder11, f"{frame_count}_person.jpg")
#                     cv2.imwrite(full,frame)
#                     # cv2.imwrite(output_path, cropped_frame)
#                     break
#             else:
#                 # box = person_data["box"]
#                 # x1, y1, x2, y2 = box
#                 # cropped_frame = frame[y1:y2, x1:x2]
#                 # output_path = os.path.join(output_folder, f"{frame_count}_person.jpg")
#                 full =  os.path.join(output_folder22, f"{frame_count}_person.jpg")
#                 cv2.imwrite(full,frame)
#                 # cv2.imwrite(output_path, cropped_frame)



#             for i , person_data in enumerate(frame_data.values()):
#                 # print(i)
#                 label = person_data["label"]
#                 if label == 1:
#                     box = person_data["box"]
#                     x1, y1, x2, y2 = box
#                     cropped_frame = frame[y1:y2, x1:x2]
#                     output_path = os.path.join(output_folder, f"{frame_count}_person{i+1}.jpg")
#                     output_path2 = os.path.join(output_folder11, f"{frame_count}_person{i+1}.jpg")
#                     cv2.imwrite(output_path, cropped_frame)

#                 else:
#                     box = person_data["box"]
#                     x1, y1, x2, y2 = box
#                     cropped_frame = frame[y1:y2, x1:x2]
#                     output_path = os.path.join(output_folder2, f"{frame_count}_person{i+1}.jpg")
#                     output_path2 = os.path.join(output_folder22, f"{frame_count}_person{i+1}.jpg")
#                     cv2.imwrite(output_path, cropped_frame)
     


#         frame_count += 1

#     # Release the video capture
#     cap.release()

#     # Close all OpenCV windows
#     cv2.destroyAllWindows()

#     return data

In [ ]:
# @app.route('/video', methods=['GET'])
# def video():
#     return send_file('C:/OpenCV/consult/output.mp4', mimetype='video/mp4')

In [ ]:
# if __name__ == '__main__':
    
#     app.run(port=5000)